# Gehalt.de by Stepstone Webscraper

## Part I
Datensammeln

In [1]:
from bs4 import BeautifulSoup
from datetime import datetime 
import pandas
import requests
from sqlalchemy import create_engine, text

In [3]:
connection = create_engine('mysql+pymysql://root:@localhost/businessintelligence')

In [5]:
with connection.begin() as con:
    jobs = pandas.read_sql(sql=text(f'''
        SELECT title, location 
        FROM job
        WHERE date = \'{datetime.now().date()}\'
    '''), con=con)
jobs

OperationalError: (pymysql.err.OperationalError) (2003, "Can't connect to MySQL server on 'localhost' ([WinError 10061] No connection could be made because the target machine actively refused it)")
(Background on this error at: https://sqlalche.me/e/14/e3q8)

In [ ]:


headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36'
}

helper = []
for index, row in jobs.iterrows():
    url = f"https://www.gehalt.de/stellenangebote-jobs/suche?searchtext={row['title']}&location={row['location']}"
    page = requests.get(url, headers)
    soup = BeautifulSoup(page.content, 'html.parser')
    try:
        salary_month_min = soup.find('div', class_ = 'gehaltMin').find('span', class_ = 'month').text.strip()
        salary_year_min = soup.find('div', class_ = 'gehaltMin').find('span', class_ = 'year').text.strip()
        salary_month_max = soup.find('div', class_ = 'gehaltMax').find('span', class_ = 'month').text.strip()
        salary_year_max = soup.find('div', class_ = 'gehaltMax').find('span', class_ = 'year').text.strip()
    except:
        salary_month_min = ''
        salary_month_max = ''
        salary_year_min = ''
        salary_year_max = ''
    salary = {
        'title': row['title'],
        'location': row['location'],
        'salary_month_min': salary_month_min,
        'salary_month_max': salary_month_max,
        'salary_year_min': salary_year_min,
        'salary_year_max': salary_year_max,
        'Date': datetime.now().date(),
        'Timestamp': datetime.now().time()
    }
    helper.append(salary)
df = pandas.DataFrame(helper)
df

,title,location,salary_month_min,salary_month_max,salary_year_min,salary_year_max,Date,Timestamp
0,SAP Inhouse Consultant / Entwickler (m|w|d),München,4.090 €,7.047 €,50.718 €,87.387 €,2023-02-21,09:46:53.429402
1,Junior SAP Consultant ABAP - IT Beratung / Sof...,Karlsruhe,4.009 €,6.616 €,49.715 €,82.035 €,2023-02-21,09:46:53.780870
2,SAP Consultants and Developer,Eschborn,3.777 €,6.239 €,46.833 €,77.369 €,2023-02-21,09:46:54.048028
3,Junior ERP/SAP Inhouse Consultant (m/w/d),Berlin,3.927 €,6.184 €,48.700 €,76.686 €,2023-02-21,09:46:54.370914
4,SAP Consultant,Region Stuttgart,3.951 €,6.104 €,48.996 €,75.689 €,2023-02-21,09:46:54.626814
...,...,...,...,...,...,...,...,...
256,European ERP Project Manager,Deutschland,3.676 €,6.300 €,45.583 €,78.114 €,2023-02-21,09:48:22.131396
257,ERP/IT Project Manager (m/f/d),Friedrichsdorf,3.963 €,5.733 €,49.145 €,71.085 €,2023-02-21,09:48:22.467395
258,ERP Application Support Manager (m/f/d),Bayreuth,3.490 €,5.278 €,43.281 €,65.449 €,2023-02-21,09:48:22.831269
259,ERP Application Support Manager (m/f/d),Hainichen,3.140 €,4.748 €,38.934 €,58.876 €,2023-02-21,09:48:23.163208


## Part II

Saving Data to a Database

In [ ]:
df.to_sql('salary', con=connection, index=False, if_exists='append')

261